In [19]:
from clit_recommender.util import flat_map, iterate_dirs
from clit_recommender import MD_ONLY


files = list(
    flat_map(
        lambda x: iterate_dirs(x, False), flat_map(iterate_dirs, iterate_dirs(MD_ONLY))
    )
)
x: str
names = set(map(lambda x: "/".join(x.split("/")[-2:]), files))


print(len(files))
print(len(names))
print(files[0])

19464
19464
/Users/niladi/workspace/seminar-kdd/data/2024-05-14/MDOnly/KORE_50_DBpedia.ttl/TextRazor/a7542e21d3d55116ca5c88cc0660f6347e54a74a244856ad7951ee5f5b15e984.json


In [2]:
from clit_recommender.clit_result import ClitResult

with open(files[0], "r") as f:
    j = f.read()
    clit_result = ClitResult.from_json(j)

In [5]:
task = clit_result.experiment_tasks[0]
task.pipeline_config

PipelineConfig(start_components=['MD1'], components={'md': [{'id': 'MD1', 'value': 'TextRazor'}], 'cg_ed': []}, example_id='md_combined_cged', end_components=['MD1'], display_name='MD + combined CG-ED', id=1, connections=[], pipeline_config_type='complex')

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

wrapper = SPARQLWrapper("http://localhost:7200/repositories/KDD")
wrapper.setReturnFormat(JSON)


wrapper.setQuery(
    """
        prefix aifb: <http://aifb.kit.edu/clit/recommender/>                 

        select ?s where { 
	        ?s a aifb:ClitMdSystem .
        } limit 100 
    """
)

try:
    ret = wrapper.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)

{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Babelfy'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/CLOCQ'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/DBpediaSpotlight'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Falcon%202.0'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/OpenTapioca'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/REL%20MD%20%28.properties%29'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/REL'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Refined%20MD%20%28.properties%29'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Spacy%20MD%20%28.properties%29'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/TagMe'}}
{'s': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/TextRazor'}}
{'s': {'type': 'uri', 'value': 'http://

In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON

wrapper = SPARQLWrapper("http://localhost:7200/repositories/KDD")
wrapper.setReturnFormat(JSON)


wrapper.setQuery(
    """
        prefix aifb: <http://aifb.kit.edu/clit/recommender/>
        prefix nif: <http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#>               

        select * where { 
            ?m nif:referenceContext <http://www.mpi-inf.mpg.de/yago-naga/aida/download/KORE50.tar.gz/AIDA.tsv/CEL01#char=0,> .
            ?m a aifb:ClitResult .
            ?m aifb:ofSystem ?s .
        } limit 100 
    """
)

try:
    ret = wrapper.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)

{'m': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Babelfy/cad137737d8ee3d1c6655f87b41d21714bec12464b06904192807f19a4b63286.json?mention=Harper&offset=67'}, 's': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Babelfy'}}
{'m': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Babelfy/cad137737d8ee3d1c6655f87b41d21714bec12464b06904192807f19a4b63286.json?mention=Romeo&offset=50'}, 's': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Babelfy'}}
{'m': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Babelfy/cad137737d8ee3d1c6655f87b41d21714bec12464b06904192807f19a4b63286.json?mention=Seven&offset=74'}, 's': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Babelfy'}}
{'m': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recommender/Babelfy/cad137737d8ee3d1c6655f87b41d21714bec12464b06904192807f19a4b63286.json?mention=Victoria&offset=10'}, 's': {'type': 'uri', 'value': 'http://aifb.kit.edu/clit/recomme

In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON

wrapper = SPARQLWrapper("http://localhost:7200/repositories/KDD", returnFormat=JSON)


wrapper.setQuery(
    """
        prefix aifb: <http://aifb.kit.edu/clit/recommender/>
        prefix nif: <http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#>               

        select (count(distinct(?m)) as ?count) 
        where { 
            ?m a nif:Context .
        }  
    """
)

try:
    ret = wrapper.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)

{'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer', 'type': 'literal', 'value': '2172'}}
